In [1]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

'/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/bin/python3.12'

In [2]:
import pandas as pd

pd.set_option("display.max_rows", 102)
from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader, TensorDataset

from src.utils import (
    load_MovieLens,
    create_test_user,
    train_test_val_split,
    seed_everything,
    UserMovieDataset,
    split_test_df,
    add_not_watched_movies,
    create_test_user_display_df,
    train,
    predict,
)
from src.models import NCF
from src.metrics import reccomendation_report

## Constants

In [3]:
DATA_FOLDER = "../../data/ml-1m/"
RANDOM_STATE = 7

In [4]:
seed_everything(RANDOM_STATE)

## Data

In [5]:
df_users, df_movies, df_ratings = load_MovieLens(DATA_FOLDER)

### Creating a test_user

In [6]:
df_users, df_ratings, new_user_id = create_test_user(
    df_users, df_ratings, [6, 16, 3192, 1461, 827, 887, 593]
)
print("Test user watch list:")
display(df_ratings[df_ratings.userId == new_user_id].merge(df_movies, on="movieId"))

Test user watch list:


,userId,movieId,rating,timestamp,name,genre
0,6040,6,5,0,Sabrina (1995),Comedy|Romance
1,6040,16,5,1,Sense and Sensibility (1995),Drama|Romance
2,6040,3192,5,2,Singles (1992),Comedy|Drama|Romance
3,6040,1461,5,3,Love and Other Catastrophes (1996),Romance
4,6040,827,5,4,Emma (1996),Comedy|Drama|Romance
5,6040,887,5,5,Singin' in the Rain (1952),Musical|Romance
6,6040,593,5,6,Pretty Woman (1990),Comedy|Romance


### Train-test split
Methodology: Last user interaction is a test item. The rest is train. Validation part is 20% of test.

In [7]:
df_train, df_test, df_val = train_test_val_split(df_ratings, df_movies, RANDOM_STATE)

Enriching test:   0%|          | 0/4832 [00:00<?, ?it/s]

Enriching val:   0%|          | 0/1209 [00:00<?, ?it/s]

### Loading additional data

In [8]:
# cos_dist = computeCosineSimilarities(df_train, "userId", "movieId", df_movies["movieId"].nunique())

# with open("../../data/cos_dist.pt", "wb") as f:
#     torch.save(cos_dist, f)

with open("../../data/cos_dist.pt", "rb") as f:
    cos_dist = torch.load(f)
popularity = torch.bincount(
    torch.tensor(df_train.movieId.values), minlength=df_movies["movieId"].nunique()
)
popularity = popularity / torch.max(popularity)

## NCF

In [9]:
seed_everything(RANDOM_STATE)
train_loader = DataLoader(
    UserMovieDataset(df_train, df_movies["movieId"].nunique()),
    batch_size=2048,
    shuffle=True,
)
val_loader = DataLoader(
    TensorDataset(torch.tensor(df_val["userId"]), torch.tensor(df_val["movieId"])),
    batch_size=4096,
    shuffle=False,
)
test_loader = DataLoader(
    TensorDataset(torch.tensor(df_test["userId"]), torch.tensor(df_test["movieId"])),
    batch_size=4096,
    shuffle=False,
)
num_users = df_users["userId"].nunique()
num_items = df_movies["movieId"].nunique()

  0%|          | 0/994175 [00:00<?, ?it/s]

In [10]:
seed_everything(RANDOM_STATE)
ncf = NCF(
    num_users,
    num_items,
    n_mlp_layers=4,
    mlp_layers_dim=32,
    mlp_kwargs={
        "activation": True,
        "dropout": True,
        "batchnorm": True,
        "dropout_rate": 0.6,
    },
)
display(ncf)

NCF(
  (user_embedding): Embedding(6041, 16)
  (item_embedding): Embedding(3883, 16)
  (mlp): Sequential(
    (MLP_layer_0): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=32, out_features=32, bias=True)
        (Activation): ReLU()
        (Dropout): Dropout(p=0.6, inplace=False)
        (BatchNorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (MLP_layer_1): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=32, out_features=32, bias=True)
        (Activation): ReLU()
        (Dropout): Dropout(p=0.6, inplace=False)
        (BatchNorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (MLP_layer_2): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=32, out_features=32, bias=True)
        (Activation): ReLU()
        (Dropout): Dropout(p=0.6, inplace=False)
        (BatchNorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=T

In [11]:
optimizer = torch.optim.Adam(ncf.parameters(), lr=3e-3)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
criterion = torch.nn.BCELoss()
n_epochs = 10
device = "cpu"

In [12]:
%%time
train(
    ncf,
    train_loader,
    optimizer,
    scheduler,
    criterion,
    n_epochs,
    val_loader=val_loader,
    df_val=df_val,
    cos_dist=cos_dist,
    popularity=popularity,
    verbose=True,
)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 0
Train loss 0.45069


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.5707),
 'NDCG @ 15': tensor(0.3635),
 'Diversity (ILD)': tensor(0.1438),
 'Novelty (EPC)': tensor(0.7175)}


...

{'Hit rate @ 15': tensor(0.5724),
 'NDCG @ 15': tensor(0.3408),
 'Diversity (ILD)': tensor(0.1444),
 'Novelty (EPC)': tensor(0.6976)}


  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 9
Train loss 0.36873


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.5790),
 'NDCG @ 15': tensor(0.3401),
 'Diversity (ILD)': tensor(0.1448),
 'Novelty (EPC)': tensor(0.6974)}
CPU times: user 17min 55s, sys: 51min, total: 1h 8min 56s
Wall time: 8min 3s


In [13]:
df_test["pred"] = predict(ncf, test_loader, verbose=False).numpy()
pred, target, pred_items = split_test_df(df_test, "userId", "movieId", "pred", "action")
reccomendation_report(pred, target, pred_items, cos_dist, popularity, k=15)

{'Hit rate @ 15': tensor(0.5586),
 'NDCG @ 15': tensor(0.3411),
 'Diversity (ILD)': tensor(0.1455),
 'Novelty (EPC)': tensor(0.6945)}

## Test user recommendations

In [14]:
df_test_user = add_not_watched_movies(new_user_id, df_test, df_train, df_movies)
test_user_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test_user["userId"]), torch.tensor(df_test_user["movieId"])
    ),
    batch_size=4096,
    shuffle=False,
)
df_test_user["pred"] = predict(ncf, test_user_loader, verbose=False).numpy()
display(create_test_user_display_df(df_test_user, df_movies, "pred"))

,userId,movieId,name,genre,pred
0,6040,523,Schindler's List (1993),Drama|War,0.601291
1,6040,1227,"Graduate, The (1967)",Drama|Romance,0.601291
2,6040,1203,"Godfather: Part II, The (1974)",Action|Crime|Drama,0.601291
3,6040,2222,Edward Scissorhands (1990),Drama|Romance,0.601291
4,6040,2588,"Rocky Horror Picture Show, The (1975)",Comedy|Horror|Musical|Sci-Fi,0.601291
5,6040,1656,Good Will Hunting (1997),Drama,0.601278
6,6040,1081,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi,0.601213
7,6040,1211,Annie Hall (1977),Comedy|Romance,0.601100
8,6040,2220,"Player, The (1992)",Comedy|Drama,0.599836
9,6040,1078,"Crying Game, The (1992)",Drama|Romance|War,0.591663
